<a href="https://colab.research.google.com/github/cbedart/EGBSL_IAEA/blob/main/EGBSL_IAEA_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<center><h1>EGBSL - IAES - Part #2</h1></center>**

---

In [ ]:
# @title Loading of prerequisites packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns; sns.set()
from sklearn.decomposition import PCA, KernelPCA
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import Image, display, clear_output, Javascript

import plotly.express as px

from sklearn import tree
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut

from sklearn.preprocessing import StandardScaler

import seaborn as sns; sns.set()
import plotly.express as px
import pydotplus

################################################################################

launch_rf_hyperparameters = 0

################################################################################

def tree_graph_to_png(tree, feature_names, class_names, png_file_to_save):
    tree_str = export_graphviz(tree, feature_names=feature_names, class_names=class_names,
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)
    graph.write_png(png_file_to_save)




np.seterr(divide='ignore', invalid='ignore')

!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/computed_desc_GR_tosave.pkl
!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/desc_GR_clustered_normalized_noNA.pkl
!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/desc_GR_PCA_53c.pkl

computed_desc_GR_tosave = pd.read_pickle("computed_desc_GR_tosave.pkl")
desc_GR_clustered_prep = pd.read_pickle("desc_GR_clustered_normalized_noNA.pkl")
desc_GR_PCA_53c = pd.read_pickle("desc_GR_PCA_53c.pkl")

print("\n\n\033[1mPrerequisites succesfully installed !\033[0m")

# <u>**Model \#1 - Multiple linear regression (quantitative)**</u>

In [ ]:
# @title Multiple linear regression model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X = desc_GR_clustered_prep.iloc[:,4:]
y = desc_GR_clustered_prep.loc[:,"pIC50"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LinearRegression(positive=True).fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


# Calculate mean squared error (for regression)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE) of Linear Regression: {mse:.2f}")

print(f"Root Mean Squared Error (RMSE) of Linear Regression: {rmse:.2f}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R2) of Linear Regression: {r2:.2f}\n\n")

for i in range(len(y_pred)):
  if abs(y_pred[i]) > 15:
    print(X_test.index[i])
    # print(y_pred + "\n")

plt.scatter(y_test, y_pred, s=10, zorder=22)
plt.axline((0, 0), slope=1, linestyle="dashed", c="black")
plt.xlim([4,11])
plt.ylim([4,11])
plt.xlabel("pIC50 theoretical")
plt.ylabel("pIC50 predicted")
plt.show()

In [ ]:
# @title Feature importance of the multiple linear regression model

coefs = pd.DataFrame(
    model.coef_, columns=["Coefficients"], index=X_train.columns
)
coefs = coefs.sort_values("Coefficients")
coefs.plot(kind="barh", figsize=(5, 35))
plt.title("Multiple linear regression model - Feature importance")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

In [ ]:
# @title Second multiple linear regression model if we remove the coefficients < 1

Xprep = desc_GR_clustered_prep.iloc[:,4:]
X = Xprep.loc[:,coefs["Coefficients"] > 1]
y = desc_GR_clustered_prep.loc[:,"pIC50"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("Only {} descriptors left\n".format(len(X.columns)))

model2 = LinearRegression(positive=True).fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred2))

# Calculate mean squared error (for regression)
mse = mean_squared_error(y_test, y_pred2)
print(f"Mean Squared Error (MSE) of Linear Regression: {mse:.2f}")

print(f"Root Mean Squared Error (RMSE) of Linear Regression: {rmse:.2f}")

r2 = r2_score(y_test, y_pred2)
print(f"R-squared (R2) of Linear Regression: {r2:.2f}\n\n")

for i in range(len(y_pred2)):
  if abs(y_pred2[i]) > 15:
    print(X_test.index[i])
    # print(y_pred2 + "\n")

plt.scatter(y_test, y_pred2, s=10, zorder=22)
plt.axline((0, 0), slope=1, linestyle="dashed", c="black")
plt.xlim([4,11])
plt.ylim([4,11])
plt.xlabel("pIC50 theoretical")
plt.ylabel("pIC50 predicted")
plt.show()


# <u>**Model \#2 - Linear Discriminant Analysis (LDA) (qualitative)**</u>

In [ ]:
# @title Separation of the dataset into 3 classes: Active/Less active/Inactive

active_picker = widgets.Dropdown(options=range(4,10), value=8, description="pIC50 limit of actives:", style=dict(description_width='initial'))
inactive_picker = widgets.Dropdown(options=range(4,10), value=6, description="pIC50 limit of inactives:", style=dict(description_width='initial'))

outs_active = widgets.Output()

def generation_active(x):
  with outs_active:
    clear_output()

    y = []
    y_color = []
    for i in desc_GR_clustered_prep.iloc[:,2]:
      if i > active_picker.value:
        y.append("Active")
        y_color.append("green")
      elif i < inactive_picker.value:
        y.append("Inactive")
        y_color.append("orange")
      else:
        y.append("LessActive")
        y_color.append("red")

    desc_GR_activity = pd.concat([desc_GR_clustered_prep.iloc[:,:4], pd.DataFrame(y, columns=["Activity"]), desc_GR_clustered_prep.iloc[:,4:]], axis = "columns")

    print(desc_GR_activity["Activity"].value_counts())

display(active_picker)
display(inactive_picker)
display(outs_active)

generation_active(active_picker.value)
active_picker.observe(generation_active, names='value')
inactive_picker.observe(generation_active, names='value')

In [ ]:
# @title Linear Discriminant Analysis with 3 classes
from sklearn.datasets import load_iris
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset (or replace this with your dataset)
y = [1 if i > active_picker.value else 0 for i in desc_GR_clustered_prep.iloc[:,2]]
desc_GR_activity = pd.concat([desc_GR_clustered_prep.iloc[:,:4], pd.DataFrame(y, columns=["Activity"]), desc_GR_clustered_prep.iloc[:,4:]], axis = "columns")

ygroup = []
ycat = []
for i in desc_GR_clustered_prep.iloc[:,2]:
  if i > active_picker.value:
    ygroup.append("Active")
    ycat.append(1)
  elif i < inactive_picker.value:
    ygroup.append("Inactive")
    ycat.append(-1)
  else:
    ygroup.append("LessActive")
    ycat.append(0)

desc_GR_activity = pd.concat([desc_GR_clustered_prep.iloc[:,:4], pd.DataFrame(ygroup, columns=["Activity_Label"]), pd.DataFrame(ycat, columns=["Activity"]), desc_GR_clustered_prep.iloc[:,4:]], axis = "columns")

X = desc_GR_activity.iloc[:,6:]
y = desc_GR_activity.loc[:,"Activity"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an LDA instance
lda = LinearDiscriminantAnalysis(store_covariance=True)

# Fit LDA on the training data
lda.fit(X_train, y_train)

# Predict the classes on the test set
y_pred = lda.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)


print("Upper pIC50 value = {}".format(active_picker.value))
print("Lower pIC50 value = {}".format(inactive_picker.value))

print(f"Accuracy of Linear Discriminant Analysis: {accuracy:.2f}")




In [ ]:
# @title Feature importance of the 3 classes Linear Discriminant Analysis

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 35))
fig.subplots_adjust(left=2, right=3, top=1, bottom=0, wspace=2)
# fig.tight_layout()

coefs1 = pd.DataFrame(lda.coef_[0], columns=["Coefficients"], index=X_train.columns)
coefs1 = coefs1.sort_values("Coefficients")

coefs2 = pd.DataFrame(lda.coef_[1], columns=["Coefficients"], index=X_train.columns)
coefs2 = coefs2.sort_values("Coefficients")

coefs3 = pd.DataFrame(lda.coef_[2], columns=["Coefficients"], index=X_train.columns)
coefs3 = coefs3.sort_values("Coefficients")

# coefs1 = coefs1.sort_values("Coefficients")
# coefs1.plot(kind="barh", figsize=(5, 15), ax=axes[1])
# plt.title("Linear Discriminant Analysis model - Feature importance")
# plt.axvline(x=0, color=".5")
# coefs2.plot(kind="barh", figsize=(5, 15), ax=axes[0])


axes[0].barh(np.arange(len(coefs1)), width=coefs1["Coefficients"]);
axes[0].set_title("Linear Discriminant Analysis model - Actives vs Others", fontdict={'fontsize':10});
axes[0].axvline(x=0, color=".5");
axes[0].yaxis.set_ticks(np.arange(len(coefs1)));
axes[0].yaxis.set_ticklabels(coefs1.index);

axes[1].barh(np.arange(len(coefs2)), width=coefs2["Coefficients"]);
axes[1].set_title("Linear Discriminant Analysis model - Inactives vs Others", fontdict={'fontsize':10});
axes[1].axvline(x=0, color=".5");
axes[1].yaxis.set_ticks(np.arange(len(coefs2)));
axes[1].yaxis.set_ticklabels(coefs2.index);

axes[2].barh(np.arange(len(coefs3)), width=coefs3["Coefficients"]);
axes[2].set_title("Linear Discriminant Analysis model - LessActives vs Others", fontdict={'fontsize':10});
axes[2].axvline(x=0, color=".5");
axes[2].yaxis.set_ticks(np.arange(len(coefs3)));
axes[2].yaxis.set_ticklabels(coefs3.index);

In [ ]:
# @title Linear Discriminant Analysis with 2 classes, 21 pIC50 limits, and Cross-Validation

print("\033[1mLDA models are currently being generated. It will take ~1 min\033[0m")

accuracies_mean = []
accuracies_sd = []
accuracies_split = []
accuracies_nbActives = []
accuracies_nbInactives = []

for pic50_limit in [j/100 for j in range(500,1001,25)]:
  ygroup2 = []
  ycat2 = []
  for i in desc_GR_clustered_prep.iloc[:,2]:
    if i > pic50_limit:
      ygroup2.append("Active")
      ycat2.append(1)
    else:
      ygroup2.append("Inactive")
      ycat2.append(0)

  desc_GR_activity2 = pd.concat([desc_GR_clustered_prep.iloc[:,:4], pd.DataFrame(ygroup2, columns=["Activity_Label"]), pd.DataFrame(ycat2, columns=["Activity"]), desc_GR_clustered_prep.iloc[:,4:]], axis = "columns")

  accuracies_nbInactives.append(desc_GR_activity2["Activity_Label"].value_counts()["Inactive"])
  accuracies_nbActives.append(desc_GR_activity2["Activity_Label"].value_counts()["Active"])

  X2 = desc_GR_activity2.iloc[:,6:]
  y2 = desc_GR_activity2.loc[:,"Activity"]

  X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
  lda2 = LinearDiscriminantAnalysis(store_covariance=True)
  lda2.fit(X_train2, y_train2)

  from sklearn.model_selection import cross_val_score
  from sklearn.model_selection import RepeatedStratifiedKFold
  y_pred2 = lda2.predict(X_test2)

  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
  scores = cross_val_score(lda2, X2, y2, scoring='accuracy', cv=cv, n_jobs=-1)


  accuracy2 = accuracy_score(y_test2, y_pred2)

  accuracies_mean.append(round(np.mean(scores), 2))
  accuracies_sd.append(round(np.std(scores), 2))
  accuracies_split.append(pic50_limit)


plt.scatter(accuracies_split, accuracies_mean)
plt.plot(accuracies_split, accuracies_mean)
plt.fill_between(accuracies_split, [i-j for i,j in zip(accuracies_mean,accuracies_sd)], [i+j for i,j in zip(accuracies_mean,accuracies_sd)], color="#0000FF11")
plt.xlabel("Limit of pIC50 between the 2 groups")
plt.ylabel("Accuracy")
plt.ylim([0.7,1.05])
plt.show()

pd.DataFrame([accuracies_mean, accuracies_sd, accuracies_nbActives, accuracies_nbInactives], index=["Accuracy mean", "Accuracy std", "Total actives", "Total inactives"], columns=accuracies_split)



## <u>**Model \#3 - Decision tree**</u>

In [ ]:
#@title Prepare the data for decision tree and random forest models

desc_GR_DTs = desc_GR_clustered_prep

pic50_limit = 8

ygroup2 = []
ycat2 = []
for i in desc_GR_DTs.iloc[:,2]:
  if i > pic50_limit:
    ygroup2.append("Active")
    ycat2.append(1)
  else:
    ygroup2.append("Less Active")
    ycat2.append(0)

desc_GR_activity2 = pd.concat([desc_GR_DTs.iloc[:,:4], pd.DataFrame(ygroup2, columns=["Activity_Label"]), pd.DataFrame(ycat2, columns=["Activity"]), desc_GR_DTs.iloc[:,4:]], axis = "columns")

X2 = desc_GR_activity2.iloc[:,6:]
y2 = desc_GR_activity2.loc[:,"Activity"]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

################################################################################

print("\nWe setup the pIC50 limit to {}, with :".format(pic50_limit))
print("- A total of {} active compounds (pIC50 < {}) - Coded as 1".format(desc_GR_activity2["Activity"].value_counts()[1], pic50_limit))
print("- A total of {} less active compounds (pIC50 > {}) - Coded as 0".format(desc_GR_activity2["Activity"].value_counts()[0], pic50_limit))

print("\n\033[1mDone !\033[0m \n\n")

In [ ]:
# @title First decision tree

dt1 = DecisionTreeClassifier()
dt1.fit(X_train2, y_train2)

y_pred_dt1 = dt1.predict(X_test2)
# dt1_confusion = pd.DataFrame(confusion_matrix(y_test2, y_pred_dt1), columns=["Less Actives", "Actives"], index=["Less Actives", "Actives"])

print("Accuracy on the train set = {}\n".format(round(dt1.score(X_train2, y_train2),3)))
print("Accuracy on the test set = {}".format(round(dt1.score(X_test2, y_test2),3)))

cvk_dt1 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores_cvk_dt1 = cross_val_score(dt1, X2, y2, scoring='accuracy', cv=cvk_dt1, n_jobs=-1)

print("Accuracy using 10-fold Cross Validation = {}\n".format(round(np.mean(scores_cvk_dt1),3)))


tree_graph_to_png(dt1, feature_names=X2.columns, class_names=["Less Actives", "Actives"], png_file_to_save='dt1.png')

print("Confusion matrix:")
fig, axn = plt.subplots(1,2, figsize=(12,3))
axn[0].grid(False) ; axn[1].grid(False)
confusion1_dt1 = ConfusionMatrixDisplay.from_estimator(dt1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[0], cmap=plt.cm.Blues, normalize=None)
confusion1_dt1.ax_.set_title("Confusion matrix, without normalization")
# confusion1_dt1
confusion2_dt1 = ConfusionMatrixDisplay.from_estimator(dt1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[1], cmap=plt.cm.Blues, normalize="true")
confusion2_dt1.ax_.set_title("Normalized confusion matrix")
# confusion2_dt1
plt.show()

dt1_acc_pr_rec = [round(dt1.score(X_train2, y_train2),3),
                  round(dt1.score(X_test2, y_test2),3),
                  round(confusion2_dt1.confusion_matrix[1,1] / (confusion2_dt1.confusion_matrix[1,1] + confusion2_dt1.confusion_matrix[0,1]), 3),
                  round(confusion2_dt1.confusion_matrix[1,1] / (confusion2_dt1.confusion_matrix[1,1] + confusion2_dt1.confusion_matrix[1,0]), 3)]

In [ ]:
# @title Display the first decision tree

# @markdown Do not hesitate to right clic >>> "Open in a new tab" to better see the tree

display(Image('dt1.png'))

## <u>**Model \#4 - Random forest**</u>

In [ ]:
# @title Build the first random forest model

# @markdown Choose the number of decision trees you will generate:
number_of_trees = 100 # @param {type:"integer"}

# @markdown Choose if you set an early termination of the trees after X splits, or just build the bigger trees you can (= None)
maximum_depth = "3" # @param ["None", 1, 2, 3, 4, 5]

if maximum_depth == "None":
  maximum_depth = None
else:
  maximum_depth = int(maximum_depth)

print("\n\033[1mUsing {} decision trees with an early termination setup at {}\033[0m\n".format(number_of_trees, maximum_depth))

rf1 = RandomForestClassifier(n_estimators = number_of_trees, max_depth = maximum_depth, random_state=42)
rf1.fit(X_train2, y_train2)

y_pred_rf1 = rf1.predict(X_test2)
rf1_confusion = pd.DataFrame(confusion_matrix(y_test2, y_pred_rf1), columns=["Less Actives", "Actives"], index=["Less Actives", "Actives"])

print("Accuracy on the train set = {}\n".format(round(rf1.score(X_train2, y_train2),3)))
print("Accuracy on the test set = {}".format(round(rf1.score(X_test2, y_test2),3)))

cvk_rf1 = RepeatedStratifiedKFold(n_splits=5, n_repeats = 3, random_state=1)
scores_cvk_rf1 = cross_val_score(rf1, X2, y2, scoring='accuracy', cv=cvk_rf1, n_jobs=-1)

print("Accuracy using 5-fold Cross Validation = {}\n".format(round(np.mean(scores_cvk_rf1),3)))

print("Confusion matrix:")
fig, axn = plt.subplots(1,2, figsize=(12,3))
axn[0].grid(False) ; axn[1].grid(False)
confusion1_rf1 = ConfusionMatrixDisplay.from_estimator(rf1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[0], cmap=plt.cm.Blues, normalize=None)
confusion1_rf1.ax_.set_title("Confusion matrix, without normalization")
# confusion1_rf1
confusion2_rf1 = ConfusionMatrixDisplay.from_estimator(rf1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[1], cmap=plt.cm.Blues, normalize="true")
confusion2_rf1.ax_.set_title("Normalized confusion matrix")
# confusion2_rf1
plt.show()

rf1_acc_pr_rec = [round(rf1.score(X_train2, y_train2),3),
                  round(rf1.score(X_test2, y_test2),3),
                  round(confusion2_rf1.confusion_matrix[1,1] / (confusion2_rf1.confusion_matrix[1,1] + confusion2_rf1.confusion_matrix[0,1]), 3),
                  round(confusion2_rf1.confusion_matrix[1,1] / (confusion2_rf1.confusion_matrix[1,1] + confusion2_rf1.confusion_matrix[1,0]), 3)]

In [ ]:
# @title Hyperparameters optimization of the random forest model

if launch_rf_hyperparameters == 1:
  raise FileExistsError('Already done')

rf_accuracies_train = []
rf_accuracies_test = []
rf_precision_test = []
rf_recall_test = []


for maximum_depth_loop in [1, 2, 3, 4, 5, None]:
  for number_of_trees_loop in [10,100,1000]:
  # for number_of_trees_loop in [10,100,1000,10000]:
    print("Currently running depth {} with {} trees".format(maximum_depth_loop, number_of_trees_loop))
    rfloop = RandomForestClassifier(n_estimators = number_of_trees_loop, max_depth = maximum_depth_loop, random_state=42)
    rfloop.fit(X_train2, y_train2)
    y_pred_rfloop = rfloop.predict(X_test2)
    rfloop_confusion = confusion_matrix(y_test2, y_pred_rfloop, normalize=None)

    rf_accuracies_train.append(round(rfloop.score(X_train2, y_train2),3))
    rf_accuracies_test.append(round(rfloop.score(X_test2, y_test2),3))
    rf_precision_test.append(round(rfloop_confusion[1,1] / (rfloop_confusion[1,1] + rfloop_confusion[0,1]), 3))
    rf_recall_test.append(round(rfloop_confusion[1,1] / (rfloop_confusion[1,1] + rfloop_confusion[1,0]), 3))


print("\n\033[1mDone !\033[0m")

In [ ]:
# @title Plot for the hyperparameters optimization

plt.figure(figsize=(10,5))
plt.plot(rf_accuracies_train, marker=".")
plt.plot(rf_accuracies_test, marker=".")
plt.plot(rf_precision_test, marker=".")
plt.plot(rf_recall_test, marker=".")
plt.title("Quality assessment metrics for RF hyperparameters optimization")
plt.legend(["Accuracy train", "Accuracy test", "Precision", "Recall"])
plt.ylim([-0.05,1.05])

labels = []
for maximum_depth_loop in [1, 2, 3, 4, 5, None]:
  # for number_of_trees_loop in [10,100,1000,10000]:
  for number_of_trees_loop in [10,100,1000]:
    labels.append("{} / {}".format(maximum_depth_loop, number_of_trees_loop))

plt.xticks(range(18), labels, rotation='vertical')
plt.show()